In [45]:
import cv2
import os

import json

from pathlib import Path
from tqdm import tqdm

def load_json(file_path):
    f = open(file_path)
    json_file = json.load(f)
    f.close()
    
    return json_file

In [2]:
from detectron2.data.datasets import register_coco_instances
from detectron2.data import DatasetCatalog, MetadataCatalog
from detectron2.data.datasets import load_coco_json
from pycocotools.coco import COCO
from copy import deepcopy

img_root = "/workspace/datasets/seg_by_patient/preprocessed/pos_cropped_patch_all_r1_r2"

coco_anno_root = "/workspace/datasets/seg_by_patient/preprocessed/anno/coco_format/"
pf = "pre_all_complete_"
all_anno = os.path.join(coco_anno_root, f"{pf}merge.json")
train_anno = os.path.join(coco_anno_root, f"{pf}train.json")
test_anno = os.path.join(coco_anno_root, f"{pf}test.json")


In [23]:
ods = load_coco_json(all_anno, img_root, "all", extra_annotation_keys=["patient_id"])
tr = load_coco_json(train_anno, img_root, "all", extra_annotation_keys=["patient_id"])
ts = load_coco_json(test_anno, img_root, "all", extra_annotation_keys=["patient_id"])



Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.


Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.


Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.



In [22]:
ods

[{'file_name': '/workspace/datasets/seg_by_patient/preprocessed/pos_cropped_patch_all_r1_r2/raw1_01.D2(30m)-1.JPG',
  'height': 1200,
  'width': 600,
  'image_id': 0,
  'annotations': [{'bbox': [285, 23, 92, 92],
    'category_id': 0,
    'segmentation': [[293.4530547170928,
      31.409565453390474,
      374.82228905263577,
      31.409565453390474,
      374.82228905263577,
      105.12405691794979,
      288.7766520182292,
      106.92197188590337]],
    'bbox_mode': <BoxMode.XYWH_ABS: 1>},
   {'bbox': [277, 234, 94, 94],
    'category_id': 0,
    'segmentation': [[285.0355360243056,
      237.270762619463,
      365.469514480745,
      242.66450752332372,
      358.9225691978378,
      325.3685812348301,
      283.16499344026204,
      319.97482151661103]],
    'bbox_mode': <BoxMode.XYWH_ABS: 1>},
   {'bbox': [269, 471, 96, 96],
    'category_id': 0,
    'segmentation': [[276.61801733151833,
      479.98920922140474,
      357.98728249289775,
      474.5954791319023,
      354.246

In [17]:
def xyxy_to_yolo(x1, y1, x2, y2, W, H):
    # clip to image bounds
    x1 = max(0.0, min(float(x1), float(W - 1)))
    y1 = max(0.0, min(float(y1), float(H - 1)))
    x2 = max(0.0, min(float(x2), float(W - 1)))
    y2 = max(0.0, min(float(y2), float(H - 1)))
    if x2 <= x1 or y2 <= y1:
        return None
    w = x2 - x1
    h = y2 - y1
    cx = x1 + w / 2.0
    cy = y1 + h / 2.0
    return (cx / W, cy / H, w / W, h / H)

def write_txt(p: Path, lines):
    p.parent.mkdir(parents=True, exist_ok=True)
    with open(p, "w", encoding="utf-8") as f:
        f.write("\n".join(lines))

In [29]:
def convert_anno(old_anno, target_folder):

    print("Converting to yolo style ...")

    for i in tqdm(old_anno):
        
        filename = os.path.basename(i["file_name"])[:-4]

        H, W = i["height"], i["width"]

        new_anno = []

        for anno in i["annotations"]:
            cls_id = anno["category_id"]
            bbox = anno["bbox"]
            tlx, tly, bw, bh = (k for k in bbox)

            brx = tlx + bw
            bry = tly + bh

            new_bbox = xyxy_to_yolo(tlx, tly, brx, bry, W, H)

            cx, cy, ww, hh = [round(v, 4) for v in new_bbox]

            new_anno.append(f"{cls_id} {cx} {cy} {ww} {hh}")

        out_txt = Path(target_folder) / f"{filename}.txt"

        write_txt(out_txt, new_anno)

    print("Convertion complete")

In [38]:
all_root = "/workspace/datasets/seg_by_patient/ultralytics/labels/all"
Path(all_root).mkdir(parents=True, exist_ok=True)

train_root = "/workspace/datasets/seg_by_patient/ultralytics/labels/train"
Path(train_root).mkdir(parents=True, exist_ok=True)

test_root = "/workspace/datasets/seg_by_patient/ultralytics/labels/test"
Path(test_root).mkdir(parents=True, exist_ok=True)

convert_anno(ods, all_root)
convert_anno(tr, train_root)
convert_anno(ts, test_root)


Converting to yolo style ...


100%|██████████| 274/274 [00:00<00:00, 6640.05it/s]


Convertion complete
Converting to yolo style ...


100%|██████████| 183/183 [00:00<00:00, 14834.90it/s]


Convertion complete
Converting to yolo style ...


100%|██████████| 69/69 [00:00<00:00, 14762.65it/s]

Convertion complete


In [39]:
print(len(os.listdir(all_root)))
print(len(os.listdir(train_root)))
print(len(os.listdir(test_root)))


274
183
69


In [43]:
69/274

0.2518248175182482

In [42]:
all_list = os.listdir(all_root)
train_list = os.listdir(train_root)
test_list = os.listdir(test_root)

for i in train_list:
    if i not in all_list:
        print(f"{i} from train NOT FOUND in all")

for i in test_list:
    if i not in all_list:
        print(f"{i} from test NOT FOUND in all")


In [37]:
train_list[0] not in all_list

False

In [77]:
def retrive_anno(key, db):
    output = []

    for k in db:
        if key in k:
            output.append(db[k])
    
    if len(output) == 0:
        print(f"'{key}' not found!")
        return None

    else:
        return output

        


In [54]:
nanno_root = "/workspace/datasets/seg_by_patient/anno"
all_anno_path = Path(nanno_root) / "all_complete_merge.json"
train_anno_path = Path(nanno_root) / "all_complete_train.json"
test_anno_path = Path(nanno_root) / "all_complete_test.json"

all_anno = load_json(all_anno_path)
train_anno = load_json(train_anno_path)
test_anno = load_json(test_anno_path)

all_fn_list = [i for i in all_anno]
train_fn_list = [i for i in train_anno]
test_fn_list = [i for i in test_anno]


In [71]:
all_anno

{'raw1_01.D2(30m)-1.JPG': {'labels': ['0',
   '0',
   '0',
   '0',
   '0',
   '0',
   '0',
   '0',
   '0',
   '0',
   '0',
   '0',
   '0',
   '0',
   '0'],
  'points': [[[106.25926208496094, 115.68518829345703],
    [207.1851806640625, 115.68518829345703],
    [210.88888549804688, 202.72222900390625],
    [109.96295928955078, 205.5]],
   [[290.5185241699219, 32.35185241699219],
    [371.0740661621094, 32.35185241699219],
    [371.0740661621094, 108.27777862548828],
    [285.8888854980469, 110.12963104248047]],
   [[444.22222900390625, 119.38888549804688],
    [514.5925903320312, 124.01851654052734],
    [514.5925903320312, 203.6481475830078],
    [439.59259033203125, 192.5370330810547]],
   [[430.3333435058594, 334.2037048339844],
    [512.74072265625, 338.8333435058594],
    [514.5925903320312, 429.5740661621094],
    [434.96295166015625, 432.3518371582031]],
   [[430.3333435058594, 573.0925903320312],
    [511.8148193359375, 575.870361328125],
    [510.8888854980469, 675.870361328125

In [69]:
for i in train_fn_list:
    if i not in all_fn_list:
        print(f"{i} from train NOT FOUND in all")

for i in test_fn_list:
    if i not in all_fn_list:
        print(f"{i} from test NOT FOUND in all")


nf_train = []
nf_test = []
nf_both = []

for i in all_fn_list:
    ptn = i[:7]

    if i not in train_fn_list and i not in test_fn_list:
        print(f"{i} from all NOT FOUND in BOTH train and test")
        if ptn not in nf_both:
            nf_both.append(ptn)

    elif i not in test_fn_list:
        print(f"{i} from all NOT FOUND in train")
        if ptn not in nf_train:
            nf_train.append(ptn)

    elif i not in test_fn_list:
        print(f"{i} from all NOT FOUND in test")
        if ptn not in nf_test:
            nf_test.append(ptn)

raw1_01.D2(30m)-1.JPG from all NOT FOUND in train
raw1_01.D2(30m)-2.JPG from all NOT FOUND in train
raw1_01.D2(30m)-3.JPG from all NOT FOUND in train
raw1_01.D2(30m)-4.JPG from all NOT FOUND in train
raw1_03.D2(30m)-1.JPG from all NOT FOUND in train
raw1_04.D2(30m)-1.JPG from all NOT FOUND in train
raw1_04.D2(30m)-4.JPG from all NOT FOUND in train
raw1_05.D2(30m)-1.JPG from all NOT FOUND in train
raw1_05.D2(30m)-2.JPG from all NOT FOUND in train
raw1_05.D2(30m)-4.JPG from all NOT FOUND in train
raw1_06.D2(30m)-1.JPG from all NOT FOUND in BOTH train and test
raw1_06.D2(30m)-3.JPG from all NOT FOUND in BOTH train and test
raw1_06.D2(30m)-4.JPG from all NOT FOUND in BOTH train and test
raw1_07.D2(30m)-1.JPG from all NOT FOUND in train
raw1_07.D2(30m)-2.JPG from all NOT FOUND in train
raw1_07.D2(30m)-3.JPG from all NOT FOUND in train
raw1_07.D2(30m)-4.JPG from all NOT FOUND in train
raw1_08.D2(30m)-1.JPG from all NOT FOUND in BOTH train and test
raw1_08.D2(30m)-2.JPG from all NOT FOUND in 

In [70]:
print(len(nf_train))
print(len(nf_test))
print(len(nf_both))


45
0
6


In [78]:
r106 = retrive_anno("raw1_06", all_anno)

r106

[{'labels': ['0',
   '0',
   '0',
   '0',
   '0',
   '0',
   '0',
   '0',
   '0',
   '0',
   '0',
   '0',
   '0',
   '0',
   '0'],
  'points': [[[91.0, 91.5], [193.0, 85.5], [191.0, 164.5], [92.0, 169.5]],
   [[263.0, 25.5], [355.0, 26.5], [357.0, 108.5], [265.0, 110.5]],
   [[435.0, 96.5], [531.0, 100.5], [529.0, 186.5], [432.0, 184.5]],
   [[81.0, 300.5], [187.0, 308.5], [184.0, 401.5], [78.0, 402.5]],
   [[261.0, 203.5], [362.0, 204.5], [360.0, 296.5], [264.0, 299.5]],
   [[431.0, 315.5], [532.0, 317.5], [535.0, 411.5], [430.0, 409.5]],
   [[74.0, 545.5], [178.0, 544.5], [177.0, 646.5], [74.0, 646.5]],
   [[263.0, 450.5], [361.0, 456.5], [359.0, 547.5], [258.0, 546.5]],
   [[424.0, 556.5], [513.0, 554.5], [513.0, 647.5], [426.0, 642.5]],
   [[258.0, 691.5], [351.0, 687.5], [350.0, 772.5], [253.0, 775.5]],
   [[85.0, 778.5], [179.0, 788.5], [180.0, 867.5], [86.0, 867.5]],
   [[425.0, 787.5], [516.0, 779.5], [514.0, 871.5], [425.0, 872.5]],
   [[431.0, 1002.5], [514.0, 984.5], [525.0,

In [74]:
nf_train

['raw1_01',
 'raw1_03',
 'raw1_04',
 'raw1_05',
 'raw1_07',
 'raw1_10',
 'raw1_11',
 'raw1_12',
 'raw1_13',
 'raw1_14',
 'raw1_15',
 'raw1_16',
 'raw1_17',
 'raw1_18',
 'raw1_19',
 'raw1_20',
 'raw1_22',
 'raw1_23',
 'raw1_24',
 'raw1_25',
 'raw1_27',
 'raw1_30',
 'raw1_31',
 'raw1_32',
 'raw2_03',
 'raw2_04',
 'raw2_05',
 'raw2_07',
 'raw2_08',
 'raw2_10',
 'raw2_11',
 'raw2_12',
 'raw2_13',
 'raw2_14',
 'raw2_16',
 'raw2_18',
 'raw2_19',
 'raw2_21',
 'raw2_22',
 'raw2_23',
 'raw2_24',
 'raw2_25',
 'raw2_30',
 'raw2_31',
 'raw2_32']

In [75]:
nf_test

[]

In [52]:
183+69

252

In [ ]:
tr_ptn = [os.path.basename(d["file_name"])[:7] for d in tr]
tr_ptn = list(dict.fromkeys(tr_ptn))

ts_ptn = [os.path.basename(d["file_name"])[:7] for d in ts]
ts_ptn = list(dict.fromkeys(ts_ptn))

print(f"Number of patients in TRAIN: {len(tr_ptn)}")
print(f"Number of images in TRAIN: {len(tr)}")

print(f"Number of patients in TEST: {len(ts_ptn)}")
print(f"Number of images in TEST: {len(ts)}")

Number of patients in TRAIN: 45
Number of images in TRAIN: 183
Number of patients in TEST: 15
Number of images in TEST: 69


In [ ]:

def load_coco(json_file, img_root):
    ds_dicts = load_coco_json(json_file, img_root)

    coco = COCO(json_file)
    ids = {img_id: coco.imgs[img_id].get("patient_id") for img_id in coco.imgs}

    out = []
    for d in ds_dicts:
        d = deepcopy(d)
        img_id = d["image_id"]
        d["patient_id"] = ids[img_id]
        out.append(d)

    return out

DATASET_NAME = "all_ds"

# DatasetCatalog.remove("all_ds")
# MetadataCatalog.remove("all_ds")
if DATASET_NAME in DatasetCatalog.list():
    DatasetCatalog.remove(DATASET_NAME)

if DATASET_NAME in MetadataCatalog.list():
    MetadataCatalog.remove(DATASET_NAME)


DatasetCatalog.register(DATASET_NAME, lambda: load_coco(all_anno, img_root))

# register_coco_instances(DATASET_NAME, {}, all_anno, img_root)

all_dicts = DatasetCatalog.get("all_ds")

In [9]:
all_dicts[0]

{'file_name': '/workspace/datasets/seg_by_patient/preprocessed/pos_cropped_patch_all_r1_r2/raw1_01.D2(30m)-1.JPG',
 'height': 1200,
 'width': 600,
 'image_id': 0,
 'annotations': [{'bbox': [285, 23, 92, 92],
   'category_id': 0,
   'segmentation': [[293.4530547170928,
     31.409565453390474,
     374.82228905263577,
     31.409565453390474,
     374.82228905263577,
     105.12405691794979,
     288.7766520182292,
     106.92197188590337]],
   'bbox_mode': <BoxMode.XYWH_ABS: 1>},
  {'bbox': [277, 234, 94, 94],
   'category_id': 0,
   'segmentation': [[285.0355360243056,
     237.270762619463,
     365.469514480745,
     242.66450752332372,
     358.9225691978378,
     325.3685812348301,
     283.16499344026204,
     319.97482151661103]],
   'bbox_mode': <BoxMode.XYWH_ABS: 1>},
  {'bbox': [269, 471, 96, 96],
   'category_id': 0,
   'segmentation': [[276.61801733151833,
     479.98920922140474,
     357.98728249289775,
     474.5954791319023,
     354.24616649897416,
     559.09743818264

In [10]:
import re
import numpy as np

records = DatasetCatalog.get(DATASET_NAME)

def get_ptn_id(r):
    # if "patient_id" in r:
    return str(r["patient_id"])

    # m = re.search(r"(patient_|P)(\d+)", r["file_name"], re.IGNORECASE)
    # return m.group(2) if m else r["image_id"]  # last resort: unique per image

groups = np.array([get_ptn_id(r) for r in records])  # one per image
idx_all = np.arange(len(records))

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


In [13]:
from sklearn.model_selection import GroupKFold

def group_kfold_indices(groups, n_splits=5, rng_seed=42):
    rng = np.random.default_rng(rng_seed)
    uniq = np.array(sorted(set(groups)))
    rng.shuffle(uniq)
    # map each sample to the shuffled order
    order = {g:i for i,g in enumerate(uniq)}
    order_idx = np.array([order[g] for g in groups])

    gkf = GroupKFold(n_splits=n_splits)
    # NOTE: gkf ignores y, uses groups only
    for tr, te in gkf.split(np.zeros_like(order_idx), groups=groups):
        yield tr, te

def register_split(name, base_records, indices):
    subset = [base_records[i] for i in indices]
    if name in DataCatalog.list():
        DatasetCatalog.remove(name)

    DatasetCatalog.register(name, lambda s=subset: deepcopy(s))

def get_patient_ids(records):
    ptn_ids = {}

    tmp = []

    idx = 0

    for i in records:
        ptn_id = i.get("patient_id")
        if ptn_id not in tmp:
            ptn_ids[idx] = ptn_id
            idx += 1
            tmp.append(ptn_id)

        
    return ptn_ids

In [19]:
outer_split = group_kfold_indices(groups, n_splits=5)

print(len(groups))

for o_fold, (outer_tr_idx, outer_te_idx) in enumerate(outer_split):
    # print(outer_tr_idx, outer_te_idx)

    inner_split = group_kfold_indices(groups[outer_tr_idx], n_splits=3)

    print(len(groups[outer_tr_idx]))

    for i_fold, (inner_tr_rel, inner_va_rel) in enumerate(inner_split):
        print(f"Test: {outer_tr_idx}")
        print(f"Train: {inner_tr_rel}")
        print(f"valid: {inner_va_rel}")

print("="*50)




274
219
Test: [  0   1   2   3   5   6   7   8   9  10  11  12  13  14  15  16  17  18
  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  40  41
  42  43  44  45  46  47  52  53  54  55  56  57  58  59  60  61  62  63
  64  68  69  70  71  72  73  74  75  76  77  78  79  80  81  82  83  84
  89  90  91  92  93  94  95  96  97  98  99 100 104 105 106 107 108 109
 111 112 113 118 119 120 121 122 123 124 125 126 127 129 130 131 132 135
 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 155
 156 157 158 159 160 165 166 167 168 169 170 171 172 173 174 178 179 180
 181 182 183 184 185 186 187 188 189 190 191 195 196 197 198 199 200 201
 202 203 204 205 206 207 208 209 210 211 212 213 215 216 220 221 222 223
 224 225 226 227 228 229 230 231 236 242 243 244 245 246 247 248 249 250
 251 252 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270
 271 272 273]
Train: [  6   7   8  12  13  14  15  20  21  30  31  32  33  34  35  36  37  39
  40  41  42  43

In [4]:
from detectron2.data.datasets import register_coco_instances
from detectron2.data import DatasetCatalog, MetadataCatalog
from detectron2.data.datasets import load_coco_json
from pycocotools.coco import COCO
from copy import deepcopy
from pathlib import Path

img_root = "/workspace/datasets/seg_by_patient/preprocessed/pos_cropped_patch_all_r1_r2"

coco_anno_root = Path("/workspace/datasets/seg_by_patient/preprocessed/anno/coco_format/")
all_anno = coco_anno_root / "pre_all_complete_merge.json"

anno = COCO(all_anno)

anno

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


In [7]:
ids = {img_id: anno.imgs[img_id].get("patient_id") for img_id in anno.imgs}

ids

{0: 'raw1_01',
 1: 'raw1_01',
 2: 'raw1_01',
 3: 'raw1_01',
 4: 'raw1_03',
 5: 'raw1_04',
 6: 'raw1_04',
 7: 'raw1_05',
 8: 'raw1_05',
 9: 'raw1_05',
 10: 'raw1_06',
 11: 'raw1_06',
 12: 'raw1_06',
 13: 'raw1_07',
 14: 'raw1_07',
 15: 'raw1_07',
 16: 'raw1_07',
 17: 'raw1_08',
 18: 'raw1_08',
 19: 'raw1_08',
 20: 'raw1_08',
 21: 'raw1_09',
 22: 'raw1_10',
 23: 'raw1_11',
 24: 'raw1_11',
 25: 'raw1_11',
 26: 'raw1_11',
 27: 'raw1_12',
 28: 'raw1_12',
 29: 'raw1_12',
 30: 'raw1_12',
 31: 'raw1_13',
 32: 'raw1_13',
 33: 'raw1_13',
 34: 'raw1_14',
 35: 'raw1_15',
 36: 'raw1_15',
 37: 'raw1_16',
 38: 'raw1_16',
 39: 'raw1_16',
 40: 'raw1_17',
 41: 'raw1_17',
 42: 'raw1_17',
 43: 'raw1_17',
 44: 'raw1_18',
 45: 'raw1_19',
 46: 'raw1_19',
 47: 'raw1_20',
 48: 'raw1_21',
 49: 'raw1_21',
 50: 'raw1_21',
 51: 'raw1_21',
 52: 'raw1_22',
 53: 'raw1_22',
 54: 'raw1_22',
 55: 'raw1_23',
 56: 'raw1_23',
 57: 'raw1_23',
 58: 'raw1_23',
 59: 'raw1_24',
 60: 'raw1_24',
 61: 'raw1_24',
 62: 'raw1_24',
 6

In [11]:
records

[{'file_name': '/workspace/datasets/seg_by_patient/preprocessed/pos_cropped_patch_all_r1_r2/raw1_01.D2(30m)-1.JPG',
  'height': 1200,
  'width': 600,
  'image_id': 0,
  'annotations': [{'bbox': [285, 23, 92, 92],
    'category_id': 0,
    'segmentation': [[293.4530547170928,
      31.409565453390474,
      374.82228905263577,
      31.409565453390474,
      374.82228905263577,
      105.12405691794979,
      288.7766520182292,
      106.92197188590337]],
    'bbox_mode': <BoxMode.XYWH_ABS: 1>},
   {'bbox': [277, 234, 94, 94],
    'category_id': 0,
    'segmentation': [[285.0355360243056,
      237.270762619463,
      365.469514480745,
      242.66450752332372,
      358.9225691978378,
      325.3685812348301,
      283.16499344026204,
      319.97482151661103]],
    'bbox_mode': <BoxMode.XYWH_ABS: 1>},
   {'bbox': [269, 471, 96, 96],
    'category_id': 0,
    'segmentation': [[276.61801733151833,
      479.98920922140474,
      357.98728249289775,
      474.5954791319023,
      354.246

In [14]:
print(records[0]['patient_id'])
print(records[1]['patient_id'])


raw1_01
raw1_01
